<a href="https://colab.research.google.com/github/Falconwatch/llm_course/blob/main/HW1/%D0%94%D0%971.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов.

За каждый метод можно получить максимум 5 баллов.
Разбалловка:
*   Воспроизводимость и читабельность кода - 1 балл.
*   Корректность метода - 1 балл.
*   Описание метода в техническом отчете - 2 балла.
*   Иновационность - 1 балл.


# 1. Информация о сабмите

Щербаков Игорь Андреевич

# 2. Технический отчет

<h2> Что было сделано</h2>
Раздел содержит испробованные методы, как попавшие в итоговое решение, так и отвергнутые после проверки

<h3> Предобработка</h3>
<li> Знаки пунктуации выкинул
<li> Стемминг алгоритмом Snowball - ок, полезно убирать лишние формы слов, часто опечатки в конце
<li> Удаление стоп слов - отказался от него в пользу взвешивания по idf (см ниже раздел Получение эмбедингов)

<h3>Модели</h3>
<li> Пробовал другие модели glove, обученне на бОльших датасетах в итоге остановился на той, что была использована на семинаре из-за веса самой модели
<li> Обучил на нашем сете FastText, пробовал разные значения размера эмбединга, окна и частотности вхождения. В итоговом решении оставил небольшое количестов эпох и +- оптимальные параметры - чтобы сократить время исполнения ноутбука. Длину вектора эмбединга выбрал равной 100, чтобы лучше стакать веткор с вектором glove (см раздел Получение эмбедингов), и не перетягивать на fasttext внимание при расчёте похожести

<h3>Получение эмбедингов</h3>
<li> Решил эмбеднг слова делать путём конкатенации эмбедингов из разных моделей. В итоговом решении стклеиваются эмбединги Glove и FastText.
<li> Для успешного стекинга вектора-эмбединги нормирую в пределах модели, так чтобы сумма элемнетов веткоров была равно 1
<li> От простой суммы или среднего для получения эмбединга предложения отказался в пользу взвешенной суммы слов. В качестве веса - значение idf слова, вычисленное на всём датасете. При расчёте весов слов в предложении idf нормируются, чтобы в сумме давать единицу. После чего эмбединг каждого слова умножается на соответсвующий вес


<h3>Поиск похожих</h3>
Алгоритм поиска похожих топиков на основе косинусного расстояния был заменнён на реализацию в FAISS, что позволило многократно ускорить поиск и повысить его релевантность.



# 3. *Code*

## 3.1 Импорт библиотек

In [1]:
#Установка бибилотек в окружение
#!pip install -q -r requirements.txt
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.8 MB/s eta 0:00:00


In [2]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

## 3.2 Загрузка данных

In [3]:
# download the data:

#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
#!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt

!wget https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt -O ./quora.txt



--2024-06-06 09:14:36--  https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33813903 (32M) [text/plain]
Saving to: ‘./quora.txt’

./quora.txt         100%[===================>]  32.25M  --.-KB/s    in 0.1s    

2024-06-06 09:14:37 (247 MB/s) - ‘./quora.txt’ saved [33813903/33813903]



In [4]:
data = np.array(list(open("./quora.txt", encoding="utf-8")))

## Предобработка данных

In [5]:
nltk.download("stopwords")
STOP_WORDS = set(stopwords.words('english'))
TOKENIZER = WordPunctTokenizer()
STEMMER = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import string
def preprocess_phrase(phrase):
  lower_phrase = phrase.lower()
  lower_phrase = lower_phrase.translate(str.maketrans('', '', string.punctuation))

  tokens = TOKENIZER.tokenize(lower_phrase)
  stemmed_tokens = [STEMMER.stem(t) for t in tokens]

  return stemmed_tokens

preprocess_phrase("Hi there!")

['hi', 'there']

In [7]:
preprocessed_data = [preprocess_phrase(d) for d in data[:]]

In [8]:
#Формируем глобальный словарь частот
vectorizer = TfidfVectorizer()

data_for_tfidf = [" ".join(ws) for ws in preprocessed_data]
vectorizer.fit_transform(data_for_tfidf)

words = vectorizer.get_feature_names_out()
idfs = vectorizer.idf_
idfs_normalised = idfs/np.max(idfs)

WORDS_IDFS = defaultdict(lambda: 1.0, {w:i for w,i in zip(words, idfs_normalised)})

print(WORDS_IDFS["what"])
print(WORDS_IDFS["is"])
print(WORDS_IDFS["dog"])
print(WORDS_IDFS["?"])


0.14412748635454461
0.1572065412341149
0.48880240698138905
1.0


In [9]:
def process_phrase_and_get_idf(phrase):
  preprocessed_phrase = preprocess_phrase(phrase)
  phrase_idfs = [WORDS_IDFS[word] for word in preprocessed_phrase]
  phrase_idfs = phrase_idfs/np.sum(phrase_idfs)
  return preprocessed_phrase, np.array(phrase_idfs)

process_phrase_and_get_idf("what is dog?")


(['what', 'is', 'dog'], array([0.18240835, 0.19896126, 0.61863039]))

## 3.3. Модель

In [12]:
GLOVE_MODEL = api.load('glove-twitter-100')
GLOVE_WORDS_IN_DICT = GLOVE_MODEL.key_to_index.keys()


from gensim.models import FastText

FT_MODEL = FastText(vector_size=100, window=3, min_count=1)  # instantiate
FT_MODEL.build_vocab(corpus_iterable=preprocessed_data)
FT_MODEL.train(corpus_iterable=preprocessed_data, total_examples=len(preprocessed_data), epochs=3)
FT_WORDS_IN_DICT = FT_MODEL.wv.key_to_index.keys()


In [13]:
def get_phrase_embedding(phrase,
                         models:list,
                         words_dicts:list):
  full_vector_size = 0
  for model in models:
    full_vector_size += model.vector_size

  empty_vector = np.zeros([full_vector_size], dtype='float32')
  word_vectors = []

  phrase_words, words_weights = process_phrase_and_get_idf(phrase)

  #перебираю слова в фразе
  for word in phrase_words:
    #сюда буду класть эмбединги по слову
    one_word_vectors = list()
    #перебираю доступные модели
    for i in range(len(models)):
      model = models[i]
      words_in_dict = words_dicts[i]

      vector_store = model.wv if type(model) is FastText else model

      if type(model) is FastText:
        vector_store = model.wv
      else:
        vector_store = model

      if word in words_in_dict:
        one_word_one_model_vector = vector_store.get_vector(word)
        one_word_one_model_vector = one_word_one_model_vector/np.sum(one_word_one_model_vector)
        one_word_vectors.append(one_word_one_model_vector)
      else:
        one_word_vectors.append(np.zeros([model.vector_size], dtype='float32'))
    one_word_vector = np.concatenate(one_word_vectors)
    word_vectors.append(one_word_vector)

  phrase_vector = np.dot(words_weights, word_vectors)
  return phrase_vector

my_models = [GLOVE_MODEL, FT_MODEL]
my_words_dicts = [GLOVE_WORDS_IN_DICT, FT_WORDS_IN_DICT]

t1 = get_phrase_embedding("What is dog?", my_models, my_words_dicts)
#t2 = get_phrase_embedding("What is dog?", GLOVE_MODEL, GLOVE_WORDS_IN_DICT)

In [14]:
# compute vector embedding for all lines in data
data_vectors = np.array([ get_phrase_embedding(l, my_models, my_words_dicts) for l in data[:]])

In [15]:
class MyDb:
    def __init__(self, sentences, sentence_embedings):
        self._dim = len(sentence_embedings[0])
        self._faiss = faiss.IndexFlatL2(self._dim )
        self._faiss.add(sentence_embedings)
        self._sentences = sentences

    def get_similar(self, query_embeddings, k=5):
        result = []
        if (len(query_embeddings) == 0) or (len(query_embeddings[0])==0):
            raise Exception("query_embeddings должно быть списком векторов-запросов для поиска похожих")
        d, i = self._faiss.search(x= query_embeddings, k = k)

        for r in i:
            r_sent = self._sentences[i]
            result.append(r_sent)
        return result

In [16]:
MDB = MyDb(data, data_vectors)

## 3.4. Применение модели

In [17]:
def find_nearest(query, my_models, my_words_dicts, k=10):
    emb = get_phrase_embedding(query, my_models, my_words_dicts)
    result = MDB.get_similar(np.array([emb]), k)
    return result

In [18]:
find_nearest("How to eat potatoes", my_models, my_words_dicts)

[array([['Can you eat potatoes raw?\n', 'Are beans potatoes?\n',
         'Why are potatoes green?\n',
         'What are the advantages/disadvantages of eating potatoes?\n',
         'How do red potatoes differ from white potatoes?\n',
         'Can guinea pigs eat potatoes?\n', 'Why is my potato green?\n',
         'What is the best way to eat raw potatoes?\n',
         'What is the right way to boil potatos for potato salad?\n',
         'Eating Healthy: Is eating mashed potatoes bad for me?\n']],
       dtype='<U1170')]

In [21]:
find_nearest("To be or not to be?", my_models, my_words_dicts)

[array([['To be or not to be?\n', 'How to talk to guys?\n',
         'Who to go to iit?\n', 'What does the "to it" refer to?\n',
         'Things to do to kill time?\n', 'To be deleted?\n',
         '? to be deleted\n', '? (To be deleted)\n',
         "What's the best album to work to?\n",
         'What is the best way to choose colleges to apply to?\n']],
       dtype='<U1170')]